In [39]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

## Previsão de avaliação dos sentimentos positivos/negativos

**Estratégia utilizada - TF-IDF**

 **O que é TF-IDF?**

> "O valor tf–idf, é uma medida estatística que tem o intuito de indicar
> a importância de uma palavra de um documento em relação a uma coleção
> de documentos ou em um corpus linguístico"
> 
> "O valor tf–idf de uma palavra aumenta proporcionalmente à medida que
> aumenta o número de ocorrências dela em um documento, no entanto, esse
> valor é equilibrado pela frequência da palavra no corpus."
> [Wikipedia](https://pt.wikipedia.org/wiki/Tf%E2%80%93idf)

TF - Frequência do termo, o numero de vezes que uma palavra aparece num determinado texto/documento. Porém esse número somente não é o suficiente para tirar uma conclusão absoluta, pois existem palavras que aparecem muito sem ter tanto peso, como a palavra "uma", "um".

IDF - Inverso da Frequência do Termo, diminui o peso das palavras que aparecem demais e aumenta das que aparecem menos.

![Processamento de texto usando TD-IDF](https://cdn-images-1.medium.com/max/1200/1*_OsV8gO2cjy9qcFhrtCdiw.jpeg)




## Processamento dos dados



In [44]:
#Lendo os dados
amazon = pd.read_table('amazon_cells_labelled.txt', names=['frase','sentimento'])
imdb = pd.read_table('imdb_labelled.txt', names=['frase','sentimento'])
yelp = pd.read_table('yelp_labelled.txt', names=['frase','sentimento'])

In [3]:
amazon.head()

,frase,sentimento
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


**Stop word e pontuações** - Stop word são palavras vazias, que não agregam ao modelo de IA e que podem ser retiradas, assim como as pontuações, iremos vetorizar as palavras num bow(bag of words) e não precisamos delas, então irei retira-las.

In [5]:
def text_pros(mess):
    
    #Retira pontuação
    nopunc = [char for char in mess if char not in string.punctuation]
    
    #Junta tudo em uma lista novamente
    nopunc = ''.join(nopunc)
    
    #Tira stopwords
    mess = [word for word in nopunc.split() if word not in stopwords.words('english')]
    
    return mess


In [6]:
amazon['frase'].head(5).apply(text_pros)

0        [So, way, plug, US, unless, I, go, converter]
1                       [Good, case, Excellent, value]
2                                     [Great, jawbone]
3    [Tied, charger, conversations, lasting, 45, mi...
4                                    [The, mic, great]
Name: frase, dtype: object

In [46]:
#Criando o bow de palavras como no head acima
#"Fitamos" e depois transformamos ele num bag of words
bow_frases = CountVectorizer(analyzer=text_pros).fit(amazon['frase'])
bow_frases = bow_frases.transform(amazon['frase'])

Uma vez com o bow, precisamos ter o "peso correto" das palavras, de acordo com sua incidência, então precisamos transforma-las novamente. 

In [48]:
#Mais uma vez precisamos "fitar" e transformar nossos dados
tfidf_frases = TfidfTransformer().fit(bow_frases)
tfidf_frases = tfidf_frases.transform(bow_frases)


## Machine Learning

Agora com os dados já no formato de TF-IDF podemos submete-los a um modelo de Inteligencia Artificial, o classificador escolhido foi o Naive Bayes, ou Teorema de Bayes que:

> "descreve a probabilidade de um evento, baseado em um conhecimento a
> priori que pode estar relacionado ao evento"
> [Wikipedia](https://pt.wikipedia.org/wiki/Teorema_de_Bayes)

Mas poderia ser outro, como um Random Forest por exemplo.

In [29]:
#"Fitando" o modelo.
sentimento_detect = MultinomialNB().fit(tfidf_frases, amazon['sentimento'])

In [30]:
#Tirando as primeiras previsões do modelo
all_predictions = sentimento_detect.predict(tfidf_frases)

In [49]:
print (classification_report(amazon['sentimento'], all_predictions))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       500
           1       0.96      0.99      0.97       500

   micro avg       0.97      0.97      0.97      1000
   macro avg       0.97      0.97      0.97      1000
weighted avg       0.97      0.97      0.97      1000



**Repetindo todo o processo para os dados do Imdb e Yelp**

In [18]:
bow_frases_imdb = CountVectorizer(analyzer=text_pros).fit(imdb['frase'])
bow_frases_imdb = bow_frases_imdb.transform(imdb['frase'])
tfidf_frases_imbd = TfidfTransformer().fit(bow_frases_imdb)
tfidf_frases_imbd = tfidf_frases_imbd.transform(bow_frases_imdb)

/home/caio/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # Remove the CWD from sys.path while we load stuff.


In [24]:
sentimento_detect_imdb = MultinomialNB().fit(tfidf_frases_imbd, imdb['sentimento'])

In [25]:
all_predictions_imdb = sentimento_detect_imdb.predict(tfidf_frases_imbd)

In [26]:
print (classification_report(imdb['sentimento'], all_predictions_imdb))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       362
           1       0.98      0.98      0.98       386

   micro avg       0.98      0.98      0.98       748
   macro avg       0.98      0.98      0.98       748
weighted avg       0.98      0.98      0.98       748



In [31]:
bow_frases_yelp = CountVectorizer(analyzer=text_pros).fit(yelp['frase'])
bow_frases_yelp = bow_frases_yelp.transform(yelp['frase'])
tfidf_frases_yelp = TfidfTransformer().fit(bow_frases_yelp)
tfidf_frases_yelp = tfidf_frases_yelp.transform(bow_frases_yelp)

/home/caio/.local/lib/python2.7/site-packages/ipykernel_launcher.py:10: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  # Remove the CWD from sys.path while we load stuff.


In [33]:
sentimento_detect_yelp = MultinomialNB().fit(tfidf_frases_yelp, yelp['sentimento'])

In [37]:
all_predictions_yelp = sentimento_detect_yelp.predict(tfidf_frases_yelp)

In [38]:
print (classification_report(yelp['sentimento'], all_predictions_yelp))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       500
           1       0.97      0.95      0.96       500

   micro avg       0.96      0.96      0.96      1000
   macro avg       0.96      0.96      0.96      1000
weighted avg       0.96      0.96      0.96      1000



## Considerações finais

**Possíveis melhoras**

 - Tenho ciencia que o bom desempenho dos algoritmos se da ao fato de
   que eles foram testados com os mesmos dados do que foram treinados.
   Tentei dividir os dados antes de treinar os modelos, porém me
   confundi em alguns momentos e acabava caindo em erros de tipo de
   dados e reshape.
 - Uso do Pipeline, com processamento de dados um pouco mais complexo
   como esse, o uso do Pipeline do  Sklearn pode ajudar bastante na
   "automação" dessas tratativas com os dados. Não utilizei por receio
   de treinar o modelo imdb com dados da amazon e vice-versa, em algum
   momento algum dado de um data frame "invadir" o outro
